# Solving Sudoku Puzzles
An exercise on the backtracking algorithm. 

### Setting up user-defined parameters
Warning: setting `dim_m = 5` and `dim_n = 5` (with `difficulty >= 0.45`) may cause the vanilla method to take forever.

In [27]:
# User parameters
dim_m = 5 # row
dim_n = 5 # col
difficulty = 0.4 # fraction of empty cells
run_vanilla_backtracking = True
run_constraint_prop_backtracking = True

### Creating the puzzle and defining some utility functions

In [28]:
from sudoku import Sudoku as sdk_benchmark
from copy import deepcopy
import numpy as np
import time

# Generate sudoku puzzle and solution using py-sudoku (as our benchmark)
dimension = dim_m * dim_n
puzzle = sdk_benchmark(dim_n, dim_m).difficulty(difficulty)

# Grab subgrid index
def get_subgrid_index(cell_row_ind, cell_col_ind):

    return cell_row_ind // dim_m, cell_col_ind // dim_n

# Get indices to the cells belonging to the same subgrid
def get_same_subgrid_indices(cell_row_ind, cell_col_ind, tolist=False):

    # Example of a 2 x 3 sudoku == 4 rows and 9 columns
    # examples of subgrid (1,1) indices = (,3), (,4), (,5), (2,), (3,)
    subgrid_row, subgrid_col = get_subgrid_index(cell_row_ind, cell_col_ind)

    actual_row_indices = subgrid_row*dim_m + np.array(range(dim_m))
    actual_col_indices = subgrid_col*dim_n + np.array(range(dim_n))

    return (actual_row_indices, actual_col_indices) if not tolist else (actual_row_indices.tolist(), actual_col_indices.tolist())

## Method 1: Vanilla backtracking

Reference: [YouTube video on a simple backtracking Sudoku solver](https://www.youtube.com/watch?v=eAFcj_2quWI&t=57s)

This uses vanilla backtracking, which is essentially a brute force method of trial-and-error.

1. Starting at the first empty cell, try a number that doesn't violate the rules (absent in row, column, and subgrid).
2. Then, go to the next empty cell, and try a number that doesn't violate the rules.
3. If the rules are violated, then return to a previous cell to try a different number.
4. Repeat until the board is complete.

This is done using the `vanilla_backtracking` method recursively. Each call of the method tries a number on a particular cell, which leads to a call of the method again on a subsequent cell. This happens until a call yields incompatible constraints (decided using the `is_valid` function), which leads to backtracking, whereby we step back into a previous cell and try a different number.

### Algorithm

In [29]:
# Check validity of candidate digit
def is_valid(board, curr_row, curr_col, candidate):
    
    # Check row constraint
    if candidate in board[curr_row]: return False

    # Check column constraint
    if candidate in [board[ind][curr_col] for ind in range(dimension)]: return False

    # Grab subgrid indices
    actual_row_indices, actual_col_indices = get_same_subgrid_indices(curr_row, curr_col, True)

    # Check subgrid constraint
    if candidate in [board[ind_row][ind_col] for ind_row in actual_row_indices for ind_col in actual_col_indices if ind_row != curr_row and ind_col != curr_col]: return False
    
    return True

# Vanilla backtracking algorithm
def vanilla_backtracking(board, row, col) -> tuple[list, bool]:
    
    # Check if current pointer is on the last row
    if row == dimension: return board, True

    # Check if current pointer is on the last column
    if col == dimension:
        return vanilla_backtracking(board, row+1, 0) # move to the next row

    # Check if current pointer contains a number already
    if board[row][col] is not None:
        return vanilla_backtracking(board, row, col+1) # move to the next column

    # Both row and column values are valid: on a valid cell with no values
    for candidate_val in range(1, dimension+1):

        # Check if current candidate is valid
        if is_valid(board, row, col, candidate_val):
            board[row][col] = candidate_val

            # Move on to the next cell
            board, solved = vanilla_backtracking(board, row, col+1)

            # Check if the next cell is satistfied
            if solved:
                return board, True
            else:
                board[row][col] = None

    return board, False # no possible candidates found, need to backtrack

### Solution

In [30]:
if run_vanilla_backtracking:
    start = time.time_ns()
    board, solved = vanilla_backtracking(deepcopy(puzzle.board), 0, 0)
    end = time.time_ns()
    
    # Display results and stats
    puzzle.show()
    print("\t##########\t")
    solution = puzzle.solve()
    print(solution)
    print("\t##########\t\n")
    if solved:
        if board == solution.board:
            print("Vanilla backtracking found IDENTICAL solution in {0} ms.".format((end - start) / 1e6))
        else:
            print("Vanilla backtracking found DIFFERENT solution in {0} ms.".format((end - start) / 1e6))
            output = sdk_benchmark(width=dim_n, height=dim_m, board=board)
            print(output)
    else:
        print("Vanilla backtracking did not find a solution.")
        output = sdk_benchmark(width=dim_n, height=dim_m, board=board)
        print(output)

Puzzle has multiple solutions
+----------------+----------------+----------------+----------------+----------------+
| 20 24 10 23 07 |    11    22    |    17    16 01 | 04    13 09    | 05    12    21 |
|    22    06 13 |    01          | 10    21 24 03 | 02 11 19       |       09 08 17 |
|    17       01 | 08 12          | 23 25 05    22 |          24 20 |    18 02       |
|       02 25 12 |    03 05 24    |    07 15    20 | 17    18 14    |    06 01 23 22 |
|    18    21 15 | 10    25    17 |    19 02       |    22          | 07 16       14 |
+----------------+----------------+----------------+----------------+----------------+
| 09    12    19 | 11       07 01 | 21 24 25 14 18 | 13 20       04 | 06          23 |
| 01 20          | 18 06    13    | 05 02 07    04 |       15 10 12 | 22 11          |
|    02 04 22 03 | 19    09       | 15 06    17    | 08 24    25 16 | 18    20 05 07 |
| 05 07 08 13 06 | 20 16    15 24 |    01 03       |    18       23 |    12 14    02 |
| 11 14    17

## Method 2: Backtracking with constraint propagation

This implements multiple features on top of vanilla backtracking:
- uses a constraint board to keep track of feasible values (instead of trying each value of the domain) for each cell,
- pre-solves the board by filling in cells that only have one feasible value on the constraint board,
- propagates constraints whenever a new number is tried in the backtracking process,
- checks for constraint violation more frequently after propagating constraints.

In a nutshell, on top of `board` which contains the actual game, we use a separate board to store the possible values for the empty cells in `board`; we call this `constraint_board`.

To ensure that constraints are properly propagated (and undone when candidate values turn out to be incorrect guesses), we keep track of them using the `constraint_removal_tracker` list. This serves as a LIFO container that tracks every number that gets removed as a constraint from a specific cell in `constraint_board`. Whenever a number is placed (which induces removals of possible numbers in other cells) in a cell, `constraint_removal_tracker` stores the removal of said number in associated cells as possible values. Likewise, when a number is deemed incorrect and is removed (which means that the previously removed possible number in associated cells should be reinstated), we use `constraint_removal_tracker` to trace and undo the removals.

In [31]:
# Place a number onto the board and update constraints associated with the cell
def place_number_with_propagation(board, constraint_board, constraint_removal_tracker, row, col, number):
    
    # Update board
    board[row][col] = number

    # Get subgrid indices
    subgrid_ind_row, subgrid_ind_col = get_subgrid_index(row, col)

    current_number_eliminations = []

    # Iterate through each other cell in the row
    for ind in range(dimension):

        # Iterate through all other cells in the row
        if ind != col:
            if number in constraint_board[row][ind]:

                # Remove the constraint
                constraint_board[row][ind].discard(number)

                # Track removal of constraint
                current_number_eliminations.append(((row, ind), number))

        # Iterate through all other cells in the column
        if ind != row:
            if number in constraint_board[ind][col]:
                
                # Remove the constraint
                constraint_board[ind][col].discard(number)

                # Track removal of constraint
                current_number_eliminations.append(((ind, col), number))

        # Iterate through all other cells in the subgrid
        curr_row = ind // dim_m + subgrid_ind_row * dim_m # current row index pointed by ind
        curr_col = ind % dim_n + subgrid_ind_col * dim_n # current column index pointed by ind

        if curr_row != row or curr_col != col:
            if number in constraint_board[curr_row][curr_col]:

                # Remove the constraint
                constraint_board[curr_row][curr_col].discard(number)

                # Track removal of constraint
                current_number_eliminations.append(((curr_row, curr_col), number))

        # example for (2,3) board and subgrid (1,1) 
        # indices=(2,3), (2,4), (2,5), (3,3), (3,4), (3,5) should correspond to ind=0, 1, 2, 3, 4, 5
        # i = ind//dim_row + subgrid_row*dim_row
        # j = ind%dim_col + subgrid_col*dim_col

    # Add the list of constraint removals associated with the current number placement to tracker
    constraint_removal_tracker.append(current_number_eliminations)

    return board, constraint_board, constraint_removal_tracker

# Remove a number from the board 
def remove_number_with_propagation(board, constraint_board, constraint_removal_tracker, row, col):

    # Update board
    board[row][col] = None

    # Reinstate constraints
    constraints_to_reinstate = constraint_removal_tracker.pop()

    for (row, col), number in constraints_to_reinstate:
        constraint_board[row][col].add(number)

    return board, constraint_board, constraint_removal_tracker

"""
Board initialization strategy:
    - Initialize `constraint_board` by filling each cell with the entire domain of possible values
    - For each filled cell of `board`, eliminate values in associated `constraint_board` cells (i.e., row, column, and subgrid)
    - After one round of elimination, check `constraint_board` for any cells with only one possibility left; if yes, fill `board` with that value
    - Repeat until no changes to `constraint_board`
"""

# Start with constraint propagation to solve easy cells first
def initialize_board_and_constraints(board, constraint_board):

    # Initialize constraint board
    constraint_board = [[set(range(1, dimension+1)) for i in range(dimension)] for j in range(dimension)]

    initialized = False

    constraint_satisfied_cell = []

    while not initialized:

        # Go through all the cells to check
        for i in range(dimension):
            for j in range(dimension):
                if board[i][j] is not None and (i,j) not in constraint_satisfied_cell:
                    
                    # Set the current constraint to be empty (required for checking length of constraint board cells later)
                    constraint_board[i][j] = set()

                    # Remove from row
                    [constraint_board[i][ind].discard(board[i][j]) for ind in range(dimension) if ind != j]

                    # Remove from column
                    [constraint_board[ind][j].discard(board[i][j]) for ind in range(dimension) if ind != i]

                    # Remove from subgrid
                    actual_row_indices, actual_col_indices = get_same_subgrid_indices(i, j)

                    [constraint_board[row_ind][col_ind].discard(board[i][j]) for row_ind in actual_row_indices for col_ind in actual_col_indices if row_ind != i and col_ind != j]

                    # Add to tracker
                    constraint_satisfied_cell.append((i,j))

        # Check if any number on the constraint board has one value left; populate board
        initialized = True

        for i in range(dimension):
            for j in range(dimension):

                # Check if there is only one possible value in the constraint board
                if len(constraint_board[i][j]) != 1: continue
                else:
                    board[i][j] = constraint_board[i][j].pop()
                    constraint_board[i][j] = set()
                    initialized = False # board is not completely initialized yet

    return board, constraint_board

# Backtracking with constraint propagation algorithm
def backtracking_with_constraint_propagation(board, row, col, constraint_board, constraint_removal_tracker, initial=False) -> tuple[list, bool, list]:

    if initial:
        board, constraint_board = initialize_board_and_constraints(board, constraint_board)

    # Check if current pointer is on the last row
    if row == dimension: return board, True, constraint_board

    # Check if current pointer is on the last column
    if col == dimension:
        return backtracking_with_constraint_propagation(board, row+1, 0, constraint_board, constraint_removal_tracker) # move to the next row

    # Check if current pointer contains a number already
    if board[row][col] is not None:
        return backtracking_with_constraint_propagation(board, row, col+1, constraint_board, constraint_removal_tracker) # move to the next column

    # Both row and column values are valid: on a valid cell with no values
    possible_values = deepcopy(constraint_board[row][col])

    for candidate_val in possible_values:
            
        # Place number with constraint propagation
        board, constraint_board, constraint_removal_tracker = place_number_with_propagation(board, constraint_board, constraint_removal_tracker, row, col, candidate_val)

        # Check if the constraints have been violated (i.e., empty sets) for current placement
        if any([len(constraint_board[r][c]) == 0 for r in range(dimension) for c in range(dimension) if board[r][c] == None]):
            board, constraint_board, constraint_removal_tracker = remove_number_with_propagation(board, constraint_board, constraint_removal_tracker, row, col)
            continue

        # Check if the constraints caused forced moves
        temp_placement_constraint_board = {(r,c): next(iter(constraint_board[r][c])) for r in range(dimension) for c in range(dimension) if len(constraint_board[r][c]) == 1}
        temp_placement_constraint_tracker = []

        while len(temp_placement_constraint_board) > 0:
            indices, val = temp_placement_constraint_board.popitem()
            board, constraint_board, constraint_removal_tracker = place_number_with_propagation(board, constraint_board, constraint_removal_tracker, indices[0], indices[1], val)

            # Check if the constraints have been violated (i.e., empty sets) for each new placement
            if any([len(constraint_board[r][c]) == 0 for r in range(dimension) for c in range(dimension) if board[r][c] == None]):
                board, constraint_board, constraint_removal_tracker = remove_number_with_propagation(board, constraint_board, constraint_removal_tracker, indices[0], indices[1])
                break
            else:
                # Keep track of locations of added numbers
                temp_placement_constraint_tracker.append(indices)

                # Check again to see if new constraints created
                temp_placement_constraint_board.update(
                    {
                        (r,c): next(iter(constraint_board[r][c]))
                        for r in range(dimension)
                        for c in range(dimension)
                        if len(constraint_board[r][c]) == 1 and not ((r,c) in temp_placement_constraint_tracker or (r,c) in temp_placement_constraint_board)
                    }
                )

        # Move on to the next cell
        board, solved, constraint_board = backtracking_with_constraint_propagation(board, row, col+1, constraint_board, constraint_removal_tracker)

        # Check if the next cell is satisfied
        if solved:
            return board, True, constraint_board
        else:
            # Remove all of the 'temp' placements induced by current placement (of candidate_val)
            for indices in temp_placement_constraint_tracker:
                board, constraint_board, constraint_removal_tracker = remove_number_with_propagation(board, constraint_board, constraint_removal_tracker, indices[0], indices[1])
            board, constraint_board, constraint_removal_tracker = remove_number_with_propagation(board, constraint_board, constraint_removal_tracker, row, col)

    return board, False, constraint_board # no possible candidates found, need to backtrack

### Solution

In [32]:
if run_constraint_prop_backtracking:
    start = time.time_ns()
    board, solved, _ = backtracking_with_constraint_propagation(deepcopy(puzzle.board), 0, 0, [], [], True)
    end = time.time_ns()
    
    # Display results and stats
    puzzle.show()
    print("\t##########\t")
    solution = puzzle.solve()
    print(solution)
    print("\t##########\t\n")
    if solved:
        if board == solution.board:
            print("Backtracking with Constraint Propagation found IDENTICAL solution in {0} ms.".format((end - start) / 1e6))
        else:
            print("Backtracking with Constraint Propagation found DIFFERENT solution in {0} ms.".format((end - start) / 1e6))
            output = sdk_benchmark(width=dim_n, height=dim_m, board=board)
            print(output)
    else:
        print("Backtracking with Constraint Propagation did not find a solution.")
        output = sdk_benchmark(width=dim_n, height=dim_m, board=board)
        print(output)

Puzzle has multiple solutions
+----------------+----------------+----------------+----------------+----------------+
| 20 24 10 23 07 |    11    22    |    17    16 01 | 04    13 09    | 05    12    21 |
|    22    06 13 |    01          | 10    21 24 03 | 02 11 19       |       09 08 17 |
|    17       01 | 08 12          | 23 25 05    22 |          24 20 |    18 02       |
|       02 25 12 |    03 05 24    |    07 15    20 | 17    18 14    |    06 01 23 22 |
|    18    21 15 | 10    25    17 |    19 02       |    22          | 07 16       14 |
+----------------+----------------+----------------+----------------+----------------+
| 09    12    19 | 11       07 01 | 21 24 25 14 18 | 13 20       04 | 06          23 |
| 01 20          | 18 06    13    | 05 02 07    04 |       15 10 12 | 22 11          |
|    02 04 22 03 | 19    09       | 15 06    17    | 08 24    25 16 | 18    20 05 07 |
| 05 07 08 13 06 | 20 16    15 24 |    01 03       |    18       23 |    12 14    02 |
| 11 14    17

It turns out that for bigger problems (4x4 onwards, with perhaps a >=0.4 difficulty), constraint propagation plays a significant role in speed, offering at least a magnitude of order in speed ups.